<a href="https://colab.research.google.com/github/htapiagroup/sistdist/blob/master/MCA_SDU_S6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistemas Distribuidos y Ubicuos II
## MCA Tercer cuatrimestre
### Horacio Tapia-McClung

## Resumen Semana 5

* arreglos multidimensionales con `numpy` y `dask`
* DataFrames con `pandas` y `dask`

* leyendo multiples archivos de texto (.csv)
* construyendo flujos retrasados
* cronometraje de operaciones con dataframes, `pandas` vs. `dask`

Cuando usar `pandas` y cuando usar `dask`?

# Analizando los viajes en taxi de la Ciudad de NY

In [0]:
# download taxi data

import os
plantilla = 'yellow_tripdata_2018-{:02d}.csv'
plantillaURL = 'wget https://s3.amazonaws.com/nyc-tlc/trip+data/'+plantilla
urls = (plantillaURL.format(k) for k in range(1,13)) # generator
for url in urls:
  os.system(url)

In [6]:
ls

sample_data/                 yellow_tripdata_2018-07.csv
yellow_tripdata_2018-01.csv  yellow_tripdata_2018-08.csv
yellow_tripdata_2018-02.csv  yellow_tripdata_2018-09.csv
yellow_tripdata_2018-03.csv  yellow_tripdata_2018-10.csv
yellow_tripdata_2018-04.csv  yellow_tripdata_2018-11.csv
yellow_tripdata_2018-05.csv  yellow_tripdata_2018-12.csv
yellow_tripdata_2018-06.csv


In [7]:
%ll -h yellow_tripdata_2018-*.csv

-rw-r--r-- 1 root 737M Aug  3  2018 yellow_tripdata_2018-01.csv
-rw-r--r-- 1 root 715M Aug  3  2018 yellow_tripdata_2018-02.csv
-rw-r--r-- 1 root 794M Aug  3  2018 yellow_tripdata_2018-03.csv
-rw-r--r-- 1 root 784M Aug  3  2018 yellow_tripdata_2018-04.csv
-rw-r--r-- 1 root 777M Aug  3  2018 yellow_tripdata_2018-05.csv
-rw-r--r-- 1 root 734M Aug  3  2018 yellow_tripdata_2018-06.csv
-rw-r--r-- 1 root 661M Feb  5 15:07 yellow_tripdata_2018-07.csv
-rw-r--r-- 1 root 661M Feb  5 15:09 yellow_tripdata_2018-08.csv
-rw-r--r-- 1 root 678M Feb  5 15:12 yellow_tripdata_2018-09.csv
-rw-r--r-- 1 root 744M Feb  5 15:14 yellow_tripdata_2018-10.csv
-rw-r--r-- 1 root 687M Feb  5 15:16 yellow_tripdata_2018-11.csv
-rw-r--r-- 1 root 689M Feb  5 15:19 yellow_tripdata_2018-12.csv


In [0]:
import pandas as pd

In [9]:
df = pd.read_csv('yellow_tripdata_2018-01.csv')
df.shape

(8759874, 17)

In [10]:

df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount'],
      dtype='object')

In [11]:
df['payment_type'].value_counts()

1    6105871
2    2598947
3      43204
4      11852
Name: payment_type, dtype: int64

Using `dask`

## Reading & cleaning files

Here you'll be working with a subset of the NYC Taxi Trip data. The first step is to use the Dask dd.read_csv() function to read multiple files at once. Dask will automatically concatenate the contents of the files into a single DataFrame. Notice that you'll use the option assume_missing=True in the call to dd.read_csv() to suppress warning messages.

Your job is to use a glob pattern containing the * character to read all of the CSV files from the taxi/ subdirectory into a single Dask DataFrame. You'll then construct a new column called 'tip_fraction' using the 'tip_amount' and 'total_amount' columns. The 'total_amount' is the sum of the fare, other fees, and the tip_amount.

## Instrucciones


 * Read all .csv files from the taxi/ directory (with a wildcard pattern *).
 * Create a column 'tip_fraction', which is the result of the 'tip_amount' divided by the difference of the 'total_amount' and 'tip_amount' columns.
 * Convert the 'tpep_dropoff_datetime' column to datetime using dd.to_datetime().
 * Create a column 'hour' using the .dt.hour attribute of the 'tpep_dropoff_datetime' column.


In [0]:
import dask.dataframe as dd
# Read all .csv files: df
df = dd.read_csv('taxi/yellow*.csv', assume_missing=True)

# Make column 'tip_fraction'
df['tip_fraction'] = df['tip_amount'] / (df['total_amount'] - df['tip_amount'])

# Convert 'tpep_dropoff_datetime' column to datetime objects
df['tpep_dropoff_datetime'] = dd.to_datetime(df['tpep_dropoff_datetime'])

# Construct column 'hour'
df['hour'] = df['tpep_dropoff_datetime'].dt.hour

## Filtering & grouping data

You have the Dask dataframe df prepared using multiple CSV files from the last exercise. It contains a subset of the 2015 yellow taxi ride data from New York City with some additional columns from preprocessing. Remember, none of the files have actually been loaded, nor has any computation been done to construct the new columns.

Your task now is to build a pipeline of computations to compute the hourly average tip fraction for each hour of the day across the entire year of data. You'll have to filter for payments of type 1 (credit card transactions) from the 'payment_type' column, group transactions using the 'hour' column, and finally aggregate the mean from the 'tip_fraction' column.

## Instrucciones

* Filter out rows where payment_type is 1 and call the resulting dataframe credit.
* Group credit using the 'hour' column and call the result 'hourly'.
* Select the 'tip_fraction' column and aggregate the mean.
* Display the data type of result.


In [0]:
# Filter rows where payment_type == 1: credit
credit = df[df.payment_type==1]

# Group by 'hour' column: hourly
hourly = credit.groupby('hour')

# Aggregate mean 'tip_fraction' and print its data type
result = hourly['tip_fraction'].mean()
print(type(result))

## Computing & plotting

Now that you've got the entire delayed pipeline prepared it's time compute and plot the result. Matplotlib has been imported for you as plt.

Warning: The execution of of this exercise is expected to be several seconds.

## Instrucciones


* Perform the computation on result and assign it to tip_frac.
* Print the type of tip_frac.
* Hit 'Submit Answer to view the plot.


In [0]:
import matplotlib.pyplot as plt
# Perform the computation
tip_frac = result.compute()

# Print the type of tip_frac
print(type(tip_frac))

# Generate a line plot using .plot.line()
tip_frac.plot.line()
plt.ylabel('Tip fraction')
plt.show()